# Data

In [1]:
import json
import torch
import pandas as pd
from tqdm import tqdm
from pyvi import ViTokenizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
import pickle
from sentence_transformers import SentenceTransformer, util
tqdm.pandas()
import json
import re
from transformers import pipeline
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator
from scipy import sparse
import sklearn
import underthesea
import os
from glob import glob
from pprint import pprint
import nltk
from underthesea import word_tokenize
from underthesea import text_normalize
from sklearn.decomposition import TruncatedSVD
import math
import torch
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [2]:
prefix = 'bm25_'
dataset = 'train_reg'

In [3]:
# https://github.com/namnv1113/UITHelper_QAS/blob/master/Dataset/train.json
data_file = open('data/'+dataset+'.json', 'r', encoding='utf-8-sig')
data = json.load(data_file)['data']

In [4]:
paragraphs = []
questions = []

In [5]:
for row in data:
    for paragraph in row['paragraphs']:
        paragraphs.append(paragraph['context'])
        for qa in paragraph['qas']:
            questions.append((qa['question'], qa['answers'][0]['text']))

### ModelForQuestionAnswering

In [6]:
tokenizer = AutoTokenizer.from_pretrained("nguyenvulebinh/vi-mrc-large")
model = AutoModelForQuestionAnswering.from_pretrained("nguyenvulebinh/vi-mrc-large")

# device gpu
qa_model = pipeline("question-answering", model=model,tokenizer=tokenizer, device=0)

### Corpus split

In [7]:
from underthesea import sent_tokenize
from tqdm import tqdm
def segment_documents(docs, max_length=6, stride=3):
    """
    Segment documents into smaller chunks
    """
    print('Segmenting documents into smaller chunks...')
    segments = []
    for doc in tqdm(docs):
        sentences = sent_tokenize(doc)
        for i in range(0, len(sentences), stride):
            segment = ' '.join(sentences[i:i+max_length])
            segments.append(segment)
    print('Number of documents:', len(docs))
    print('Number of segments:', len(segments))
    return segments

In [8]:
corpus_split_df=segment_documents(paragraphs)

Segmenting documents into smaller chunks...


100%|██████████| 75/75 [00:00<00:00, 1818.40it/s]

Number of documents: 75
Number of segments: 122


In [9]:
def compute_exact_match(prediction, truth):
    return int(prediction == truth)

def compute_f1(prediction, truth):
    pred_tokens = prediction.split()
    truth_tokens = truth.split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [10]:
import numpy as np

In [11]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [doc.split(" ") for doc in corpus_split_df]
bm25 = BM25Okapi(tokenized_corpus)

def get_bm25_score(query, corpus_split_df, bm25):
    tokenized_query = query.split(" ")
    doc_scores = bm25.get_scores(tokenized_query)
    return doc_scores

In [14]:
test = qa_model(question=questions[0][0], context=corpus_split_df[0], topk=5)
test


c:\Users\ngoph\.conda\envs\seminar\lib\site-packages\transformers\pipelines\question_answering.py:325: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)


[{'score': 5.1167598940082826e-06,
  'start': 70,
  'end': 96,
  'answer': 'DELF_B1 hoặc TCF_niveau_3.'},
 {'score': 2.61710647464497e-06, 'start': 70, 'end': 77, 'answer': 'DELF_B1'},
 {'score': 7.453017758507485e-08,
  'start': 70,
  'end': 96,
  'answer': 'DELF_B1 hoặc TCF_niveau_3.'},
 {'score': 4.069461922995288e-08,
  'start': 70,
  'end': 96,
  'answer': 'DELF_B1 hoặc TCF_niveau_3.'},
 {'score': 1.69395963922625e-08,
  'start': 46,
  'end': 96,
  'answer': 'sinh viên cần tối thiểu DELF_B1 hoặc TCF_niveau_3.'}]

In [13]:
asdsda

NameError: name 'asdsda' is not defined

In [ ]:
# score = w0 \* retrieval_score + w1 \* predict_score (linear regression)

results = []
for q in tqdm(questions):
  test_question = q[0]
  scores = get_bm25_score(test_question, corpus_split_df, bm25)
  idx = np.argsort(scores)[::-1][:10]
  temp_result = []
  for i in idx:
    retrieval_score = scores[i]
    result = qa_model({'question': test_question, 'context': corpus_split_df[i]})
    # rename result['score'] to predict_score
    result['retrieval_score'] = retrieval_score
    result['predict_score'] = result['score']
    result['context'] = corpus_split_df[i]
    del result['score']
    temp_result.append(result)
  results.append({
      'question': test_question,
      'answers': temp_result,
      'true_answer': q[1]
  })

In [ ]:
results[0]

In [ ]:
#save data
import pickle

with open('data/'+prefix+dataset+'_results.pkl', 'wb') as f:
    pickle.dump(results, f)

In [ ]:
#   w0 \* retrieval_score + w1 \* predict_score (linear regression)
#   w0 + w1= 1

test = results[0]
test_answers= test['answers']
w0 = 0.5
w1 = 1 - w0

for answer in test_answers:
    answer['score'] = w0 * answer['retrieval_score'] + w1 * answer['predict_score']

test_answers = sorted(test_answers, key=lambda x: x['score'], reverse=True)
best_answer = test_answers[0]
print('Question:', test['question'])
print('Predicted answer:', best_answer['answer'])
print('True answer:', test['true_answer'])
print('exact_match:', compute_exact_match(best_answer['answer'], test['true_answer']))
print('f1:', compute_f1(best_answer['answer'], test['true_answer']))

In [ ]:
# find best w0, w1
def compute_score(w0, w1, results):
    exact_match = 0
    f1 = 0
    for test in results:
        test_answers= test['answers']
        for answer in test_answers:
            answer['score'] = w0 * answer['retrieval_score'] + w1 * answer['predict_score']
        test_answers = sorted(test_answers, key=lambda x: x['score'], reverse=True)
        best_answer = test_answers[0]
        exact_match += compute_exact_match(best_answer['answer'], test['true_answer'])
        f1 += compute_f1(best_answer['answer'], test['true_answer'])
    f1 = f1 / len(results)
    return exact_match, f1

exact_match, f1 = compute_score(0.5, 0.5, results)
print('exact_match:', exact_match)
print('f1:', f1)

# find best w0, w1
best_w0 = 0
best_w1 = 0
best_exact_match = 0
best_f1 = 0
for w0 in np.arange(0, 1, 0.001):
    for w1 in np.arange(0, 1, 0.001):
        if w0 + w1 == 1:
            exact_match, f1 = compute_score(w0, w1, results)
            if exact_match > best_exact_match:
                best_w0 = w0
                best_w1 = w1
                best_exact_match = exact_match
                best_f1 = f1
print()
print('best_w0:', best_w0)
print('best_w1:', best_w1)
print('best_exact_match:', best_exact_match)
print('best_f1:', best_f1)

In [ ]:
test = results[0]
test_answers= test['answers']

for answer in test_answers:
    answer['score'] = best_w0 * answer['retrieval_score'] + best_w1 * answer['predict_score']

test_answers = sorted(test_answers, key=lambda x: x['score'], reverse=True)
best_answer = test_answers[0]
print('Question:', test['question'])
print('Predicted answer:', best_answer['answer'])
print('True answer:', test['true_answer'])
print('exact_match:', compute_exact_match(best_answer['answer'], test['true_answer']))
print('f1:', compute_f1(best_answer['answer'], test['true_answer']))

In [ ]:
print(dataset)
print('number of docs:',len(paragraphs))
print('number of segments:',len(corpus_split_df))
print('number of questions:',len(results))
# print('Exact Match: {} ({})'.format(result['exact_match'].sum()/len(result), result['exact_match'].sum()))
# print('F1: ', result['f1'].mean())